In [1]:
import math

import numpy
import scipy


def black_scholes(spot_price, strike_price, risk_rate, mature_time, volitility, callput):
    divisor = volitility * mature_time * math.sqrt(mature_time)
    d1 = (math.log(spot_price / strike_price) + (risk_rate + ((volitility ** 2) / 2)) * mature_time) / (divisor)
    d2 = (d1 - divisor)
    call = (spot_price * scipy.stats.norm.cdf(d1)) - strike_price * math.exp(
        -risk_rate * mature_time) * scipy.stats.norm.cdf(d2)
    put = strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(
        -d2) - spot_price * scipy.stats.norm.cdf(-d1)
    if callput == "call":
        return call
    else:
        return put
    # call = (spot_price * scipy.stats.norm.cdf((math.log(spot_price/strike_price)+(risk_rate+((volitility**2)/2))*mature_time) / (volitility*math.sqrt(mature_time)))) - strike_price * math.exp(-risk_rate * mature_time) * scipy.stats.norm.cdf(((math.log(spot_price/strike_price)+(risk_rate+((volitility**2)/2))*mature_time) / (volitility*math.sqrt(mature_time)) - (volitility * math.sqrt(mature_time))))


print(black_scholes(100, 120, 0.05, 0.5, 0.2, "call"))
print(black_scholes(100, 120, 0.05, 0.5, 0.2, "put"))
print(black_scholes(100, 96, 0.05, 0.5, 0.2, "call"))
print(black_scholes(100, 96, 0.05, 0.5, 0.2, "put"))

0.03311723312069925
17.070306676520616
7.009859140996113
0.6396106957160494


In [2]:
def binomial(starting_value, strike, rate, volatility, time, steps, option_type, exercise):
    up = math.exp(volatility * math.sqrt(time / steps))
    down = 1 / math.exp(volatility * math.sqrt(time / steps))
    rate_value = math.exp((-rate) * time / steps)
    increase_probability = ((math.exp(rate * (time / steps)) - down) / (up - down))
    decrease_probability = 1 - increase_probability
    tree = [[0]]
    tree[0][0] = tree[0][0] + starting_value
    for cur_depth in range(1, steps + 1):
        new_values = []
        old = tree[-1]
        for child in range(0, cur_depth + 1):
            if cur_depth == child:
                new = old[-1] * down
            elif child == 0:
                new = old[0] * up
            else:
                new = ((old[child - 1] * up) + (old[child] * down)) / 2
            new_values.append(new)
        tree.append(new_values)
    children = tree[-1]
    values = []
    for i in children:
        if option_type == "put":
            if strike - i > 0:
                option = strike - i
            else:
                option = 0
        else:
            if i - strike > 0:
                option = i - strike
            else:
                option = 0
        values.append(option)
    k = steps
    while k >= 0:
        for j in range(k):
            values[j] = (increase_probability * values[j + 1] + decrease_probability * values[j]) * rate_value
            if exercise == "american":
                if option_type == "put":
                    exercise_value = strike - children[j]
                else:
                    exercise_value = children[j] - strike
                values[j] = max(values[j], exercise_value)
        k = k - 1

    return values[0]


print(binomial(100, 120, 0.05, 0.2, 0.5, 183, "put", "europiean"))
print(binomial(100, 120, 0.05, 0.2, 0.5, 183, "call", "europiean"))
print(binomial(100, 120, 0.05, 0.2, 0.5, 183, "put", "american"))
print(binomial(100, 120, 0.05, 0.2, 0.5, 183, "call", "american"))
print(binomial(100, 96, 0.05, 0.2, 0.5, 183, "put", "europiean"))
print(binomial(100, 96, 0.05, 0.2, 0.5, 183, "call", "europiean"))
print(binomial(100, 96, 0.05, 0.2, 0.5, 183, "put", "american"))
print(binomial(100, 96, 0.05, 0.2, 0.5, 183, "call", "american"))

17.470273425653982
1.095140270757515
17.470273425653982
557.4141527783545
2.665178842348572
9.697483576132093
2.665178842348572
581.4141527783545


In [3]:
def monte_carlo(stock_price, strike_price, time, interest_rate, volitility, type, steps, simulations):
    volitility *= time
    st = stock_price * numpy.exp(numpy.cumsum(((interest_rate - 0.5 * volitility * volitility) * time / steps) + (
                volitility * numpy.sqrt(time / steps)) * numpy.random.normal(size=(steps, simulations)), axis=0))
    if type == "call":
        values = numpy.maximum(st[-1] - strike_price, 0)
    else:
        values = numpy.maximum(strike_price - st[-1], 0)
    return numpy.mean(values) * numpy.exp(-interest_rate * time)


print(monte_carlo(100, 120, 0.5, 0.05, 0.2, "call", 500, 200000))
print(monte_carlo(100, 120, 0.5, 0.05, 0.2, "put", 500, 200000))
print(monte_carlo(100, 96, 0.5, 0.05, 0.2, "call", 500, 200000))
print(monte_carlo(100, 96, 0.5, 0.05, 0.2, "put", 500, 200000))

0.03597258105720598
17.075799439816848
7.028675875946282
0.6465759600824077
